In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch3d as p3d
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.io import load_obj, load_ply, save_ply
from pytorch3d.loss import chamfer_distance

In [4]:
### config

# 定义点云数据的输入维度和输出维度
input_dim = 3  # 每个点的特征维度
output_dim = 3  # 输出点云的特征维度（可以与输入维度相同）

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 20
batch_size = 40

sxxxxes = os.listdir("../../data/all_results/")

In [13]:
# 创建一个简单的全连接神经网络类
class PointCloudFCNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PointCloudFCNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # 输入层到隐藏层1
        self.fc2 = nn.Linear(64, 128)  # 隐藏层1到隐藏层2
        self.fc3 = nn.Linear(128, 256)  # 隐藏层2到隐藏层3
        self.fc4 = nn.Linear(256, 128)  # 隐藏层3到隐藏层4
        self.fc5 = nn.Linear(128, output_dim)  # 隐藏层4到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)  # 输出层不使用激活函数
        return x


# DataSet类
class TrainingSet(Dataset):
    def __init__(self, s_name_list, transform=None, sample_num=5000) -> None:
        self.input = [
            f"../../data/all_results/{sname}/0_sphere.obj" for sname in s_name_list
        ]
        self.target = [
            f"../../data/all_results/{sname}/{sname}.ply" for sname in s_name_list
        ]
        self.sample_num = sample_num
        self.transform = transform

    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        #对input采样5000个点
        input_tensor_points, input_tensor_faces, _ = load_obj(self.input[index], load_textures=False)
        input_mesh = Meshes([input_tensor_points], [input_tensor_faces.verts_idx]) #注意这里有个.verts_idx
        input_tensor_points_sampled = sample_points_from_meshes(input_mesh,5000).squeeze(0)
        #对output采样5000个点
        target_tensor_points, target_tensor_faces = load_ply(self.target[index])
        target_mesh = Meshes([target_tensor_points],[target_tensor_faces])

        return input_tensor_points_sampled.to(device),target_tensor_points.to(device)

    def get_target(self, input):
        raise NotImplemented

In [11]:
#################################################################################
pts,faces = load_ply(f"../../data/all_results/s0004_pulmonary_artery.nii.g_1/s0004_pulmonary_artery.nii.g_1.ply")
mymesh = Meshes([pts],[faces])
a = sample_points_from_meshes(mymesh,5)
a = a.squeeze(0)
a
# mymesh
#################################################################################

tensor([[ 0.0352, -0.2595,  0.0741],
        [-0.1906, -0.1780, -0.0342],
        [ 0.5646, -0.1861, -0.0104],
        [ 0.0412, -0.2513,  0.1405],
        [-0.3935, -0.5400, -0.1011]])

In [15]:
def plot_pointcloud(points, title=""):
    """Sample points uniformly from the surface of the mesh."""
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, z, -y)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [16]:
mydataset = TrainingSet(sxxxxes)
data_loader = DataLoader(mydataset, batch_size=batch_size, shuffle=False)


In [17]:
# 创建模型实例
model = PointCloudFCNet(input_dim, output_dim).to(device)
model

PointCloudFCNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=3, bias=True)
)

In [18]:
# 定义损失函数和优化器
criterion = chamfer_distance
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器

In [19]:
next(iter(data_loader))

RuntimeError: stack expects each tensor to be equal size, but got [7088, 3] at entry 0 and [10314, 3] at entry 1

In [ ]:
# 训练模型

for epoch in range(num_epochs):
    for input_point_cloud,target_point_cloud in data_loader:
        optimizer.zero_grad()
        output_point_cloud = model(input_point_cloud)  # 前向传播
        loss,_ = criterion(output_point_cloud, target_point_cloud)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 优化模型参数

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    # if (epoch + 1) % 5 == 0:
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # plot_pointcloud(output_point_cloud)# 绘制点云

Epoch [1/20], Loss: 0.0081


KeyboardInterrupt: 

In [ ]:
# 使用训练好的模型生成新的点云数据
new_point_cloud = model(input_point_cloud) # TODO: 这里要改成用路径读一个点云
new_point_cloud
# 输出的new_point_cloud包含了经过神经网络处理后的新点云数据

In [ ]:
#看一眼新的输出长啥样
# save_ply("./kkwd.ply",new_point_cloud.squeeze())